In [44]:
import sqlite3
from pathlib import Path
import os
import pandas as pd

# Determine project base (walk up until pyproject.toml / uv.lock / .git)
nb_dir = Path.cwd()
base = nb_dir
markers = ("pyproject.toml", "uv.lock", ".git")
while base != base.parent and not any((base / m).exists() for m in markers):
    base = base.parent

# Build candidate paths (env first)
env_path = os.environ.get("DB_PATH")
candidates = []
if env_path:
    candidates.append(Path(env_path))

candidates += [
    base / "data" / "thermo_data.db",
    nb_dir / "data" / "thermo_data.db",
    base / "thermo_data.db",
    nb_dir / "thermo_data.db",
]

resolved = next((p for p in candidates if p and Path(p).exists()), None)
if not resolved:
    raise FileNotFoundError(
        f"thermo_data.db not found. Tried: {[str(p) for p in candidates]}"
    )

db_path = str(resolved)
print(f"Using database: {db_path}")

# Create a connection to the SQLite database
conn = sqlite3.connect(db_path)

Using database: c:\IDE\repository\agents_for_david\data\thermo_data.db


In [45]:
# Получение информации о структуре таблицы compounds
import sqlite3
import pandas as pd

# Подключение к базе данных
conn = sqlite3.connect(db_path)

# Получение информации о колонках
cursor = conn.cursor()
cursor.execute("PRAGMA table_info(compounds)")
columns_info = cursor.fetchall()

print("Структура таблицы compounds:")
for col in columns_info:
    print(f"  {col[1]} ({col[2]}) - NOT NULL: {bool(col[3])}, DEFAULT: {col[4]}, PK: {bool(col[5])}")

print("\nПервые 5 записей:")
df_sample = pd.read_sql_query("SELECT * FROM compounds LIMIT 5", conn)
print(df_sample)

print(f"\nВсего записей в таблице: {pd.read_sql_query('SELECT COUNT(*) as count FROM compounds', conn)['count'].iloc[0]}")

# conn.close()

Структура таблицы compounds:
  Formula (TEXT) - NOT NULL: False, DEFAULT: None, PK: False
  Structure (TEXT) - NOT NULL: False, DEFAULT: None, PK: False
  FirstName (TEXT) - NOT NULL: False, DEFAULT: None, PK: False
  SecondName (TEXT) - NOT NULL: False, DEFAULT: None, PK: False
  Phase (TEXT) - NOT NULL: False, DEFAULT: None, PK: False
  CAS (TEXT) - NOT NULL: False, DEFAULT: None, PK: False
  MeltingPoint (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  BoilingPoint (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  Density (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  Solubility (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  Color (INTEGER) - NOT NULL: False, DEFAULT: None, PK: False
  H298 (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  S298 (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  Tmin (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  Tmax (REAL) - NOT NULL: False, DEFAULT: None, PK: False
  f1 (REAL) - NOT NULL: False, DEFAULT: None, PK: Fal

In [46]:
import sqlite3
import pandas as pd

# ====================================
# ИСПРАВЛЕННЫЙ SQL ЗАПРОС ДЛЯ ПОИСКА TiO2:
# ====================================
# Проблема была в неправильных кавычках в LIKE паттернах
test_sql_query = """
SELECT * FROM compounds WHERE (TRIM(Formula) = 'AgNO3' OR Formula LIKE 'AgNO3(%' OR TRIM(Formula) = 'AgNO2' OR Formula LIKE 'AgNO2(%' OR TRIM(Formula) = 'NO' OR Formula LIKE 'NO(%' OR TRIM(Formula) = 'O2' OR Formula LIKE 'O2(%') LIMIT 10000
"""

print("=== ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО SQL ЗАПРОСА ===")
print(f"Запрос: {test_sql_query.strip()}")
print()

try:
    df = pd.read_sql_query(test_sql_query, conn)
    
    print("✅ Запрос выполнен успешно!")
    # Фильтрация по диапазону температур 400-600 K (полное или частичное пересечение интервалов [Tmin, Tmax])
    low = 673
    high = 673
    # Если колонка отсутствует — создаём Series из NaN той же длины
    if 'Tmin' in df.columns:
        tmin = pd.to_numeric(df['Tmin'], errors='coerce')
    else:
        tmin = pd.Series([pd.NA] * len(df), index=df.index)
    if 'Tmax' in df.columns:
        tmax = pd.to_numeric(df['Tmax'], errors='coerce')
    else:
        tmax = pd.Series([pd.NA] * len(df), index=df.index)
    # Заполняем NaN для логики пересечения: отсутствующий Tmin -> -inf, отсутствующий Tmax -> +inf
    tmin_f = tmin.fillna(-float('inf'))
    tmax_f = tmax.fillna(float('inf'))
    # Условие пересечения отрезков: не (Tmax < low или Tmin > high)
    mask = (~(tmax_f < low)) & (~(tmin_f > high))
    df_filtered = df[mask]
    print(f"Записей всего: {len(df)}, после фильтрации по 400-600 K: {len(df_filtered)}")
    display(df_filtered)

except Exception as e:
    print(f"❌ ОШИБКА ВЫПОЛНЕНИЯ ЗАПРОСА:")
    print(f"   {type(e).__name__}: {e}")
    print()
    print("🔍 ПРОВЕРКА СИНТАКСИСА:")
    print("Убедитесь что:")
    print("  - Кавычки правильно экранированы")
    print("  - Нет лишних символов") 
    print("  - Структура WHERE корректна")

finally:
    conn.close()

print()
print("=== ЗАВЕРШЕНИЕ ТЕСТИРОВАНИЯ ===")

=== ТЕСТИРОВАНИЕ ИСПРАВЛЕННОГО SQL ЗАПРОСА ===
Запрос: SELECT * FROM compounds WHERE (TRIM(Formula) = 'AgNO3' OR Formula LIKE 'AgNO3(%' OR TRIM(Formula) = 'AgNO2' OR Formula LIKE 'AgNO2(%' OR TRIM(Formula) = 'NO' OR Formula LIKE 'NO(%' OR TRIM(Formula) = 'O2' OR Formula LIKE 'O2(%') LIMIT 10000

✅ Запрос выполнен успешно!
Записей всего: 1702, после фильтрации по 400-600 K: 163


,Formula,Structure,FirstName,SecondName,Phase,CAS,MeltingPoint,BoilingPoint,Density,Solubility,...,Tmin,Tmax,f1,f2,f3,f4,f5,f6,ReliabilityClass,Reference
8,NO(g),None,Nitrogen oxide,None,g,10102-43-9,112.00,122.00,0.000,0.0,...,298.15,1300.0,23.046150,15.483989,2.335560,-4.781010,0.0,0.0,1,Glushko 94; Landolt 01
17,NO(+g),None,Nitrosyl +ion,Nitrogen oxide +ion,g,14452-93-8,0.00,0.00,0.000,0.0,...,298.15,2000.0,24.358338,10.266860,1.678287,-2.198194,0.0,0.0,1,JANAF 85; Belov 99
21,O2(g),None,Oxygen,None,g,7782-44-7,54.36,90.20,0.000,0.0,...,298.15,700.0,22.059550,20.886720,1.620555,-8.207332,0.0,0.0,1,Nasa 93; Landolt 99
37,O2(0.01barg),None,Oxygen,None,g,7782-44-7,0.00,61.29,0.000,0.0,...,430.00,700.0,20.534711,24.770200,2.257450,-10.918550,0.0,0.0,1,Lemmon 02
48,O2(0.05barg),None,Oxygen,None,g,7782-44-7,0.00,68.79,0.000,0.0,...,520.00,700.0,21.100661,23.519369,1.921230,-10.145119,0.0,0.0,1,Lemmon 02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1692,NO(+g),None,Nitrogen oxide,None,g,None,0.00,0.00,0.000,0.0,...,600.00,2600.0,28.191761,6.200681,-5.121210,-1.121311,0.0,0.0,1,JANAF 85
1694,O2(g),None,Oxygen,None,g,None,54.60,90.00,1.429,0.0,...,298.15,5000.0,31.321390,3.895300,-3.104525,-0.334720,0.0,0.0,1,Barin 93
1697,O2(-g),None,Oxygen -ion,None,g,None,0.00,0.00,0.000,0.0,...,600.00,6000.0,37.199900,0.531367,-11.798866,0.000000,0.0,0.0,1,JANAF 85
1699,O2(-2g),None,Peroxide -2 ion,None,g,None,0.00,0.00,0.000,0.0,...,298.15,1000.0,26.179258,18.187828,-0.506263,-8.468407,0.0,0.0,2,"Karapet 70, Loewen 87"



=== ЗАВЕРШЕНИЕ ТЕСТИРОВАНИЯ ===
